In [127]:
import requests
from utils import get_gitacs_token # for access token

In [135]:
token = get_gitacs_token()
headers = {'Authorization': f'token {token}'}

url = 'https://api.github.com/search/users'
params = {
    'q': 'location:Berlin followers:>200',
    'per_page': 100,
    'sort': 'followers',
    'order': 'desc'
}

In [136]:
all_users = []

while True:
    response = requests.get(url, headers=headers, params=params)
    users = response.json()['items']
    all_users.extend(users)

    # Check if there is a 'next' page
    if 'next' in response.links:
        url = response.links['next']['url']  
    else:
        break

In [137]:
len(all_users)

603

In [131]:
# create users data
from tqdm import tqdm
# import time
list_user = []
# cnt = 0
for ul in tqdm(all_users):
    # cnt += 1
    # print(f"Iteration {cnt}")
    
    # # Wait for 1 minute after every 10th iteration
    # if cnt % 60 == 0:
    #     print("Waiting for 1 minute")
    #     time.sleep(60)

    lop_resp = requests.get(ul['url'], headers=headers )
    user_json = lop_resp.json() 

    # clean up company name
    if user_json["company"]:
        company = user_json["company"].strip().lstrip('@').upper()
    else: company = ""     

    # create user data list for each entry
    list_user.append({'login': user_json['login'], 'name': user_json['name'], 'company': company,
                      'location': user_json['location'], 'email':user_json['email'], 
                      'hireable': user_json['hireable'], 'bio': user_json['bio'], 'public_repos':user_json['public_repos'], 
                      'followers':user_json['followers'], 'following':user_json['following'], 'created_at':user_json['created_at']})



100%|██████████| 610/610 [03:41<00:00,  2.75it/s]


In [132]:
# create user data frame
import pandas as pd

df_user_data = pd.DataFrame(list_user)

# convert to boolean
df_user_data['hireable'] = df_user_data['hireable'].astype(bool) 

In [133]:
df_user_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 610 entries, 0 to 609
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   login         610 non-null    object
 1   name          600 non-null    object
 2   company       610 non-null    object
 3   location      610 non-null    object
 4   email         329 non-null    object
 5   hireable      610 non-null    bool  
 6   bio           440 non-null    object
 7   public_repos  610 non-null    int64 
 8   followers     610 non-null    int64 
 9   following     610 non-null    int64 
 10  created_at    610 non-null    object
dtypes: bool(1), int64(3), object(7)
memory usage: 48.4+ KB


In [134]:
# write as csv
df_user_data.to_csv('df_user_data.csv',index=False)

In [53]:
# create repository data

# get repositories functions
def get_repos(repos_url):
    list_repos = []
    page = 1
    while True:
        url_rep = f"{repos_url}?sort=pushed&direction=desc&page={page}&per_page=100"
        response = requests.get(url_rep, headers=headers)
        json_rep = response.json()
        if json_rep:
            list_repos.extend(json_rep)
            if len(json_rep) < 100:
                break
            page += 1
        else:
            break
    return list_repos


In [112]:
len(list_repo_data)

20405

In [109]:
# Fetch repository data -- 391
list_repo_data = []
for ul in tqdm(all_users[391:]):
    list_repos = get_repos(ul['repos_url'])
    for rp in list_repos[:500]:  # Fetch up to 500 most recently pushed repositories
        list_repo_data.append({
            'login': ul['login'],
            'full_name': rp.get('full_name', ''),
            'created_at': rp.get('created_at', ''),
            'stargazers_count': rp.get('stargazers_count', 0),
            'watchers_count': rp.get('watchers_count', 0),
            'language': rp.get('language', ''),
            'has_projects': rp.get('has_projects', False),
            'has_wiki': rp.get('has_wiki', False),
            'license_name': rp['license'].get('key', '') if rp['license'] else ""
        })

100%|██████████| 218/218 [10:24<00:00,  2.86s/it]


In [110]:
# store repo data in data frame
# df_repos = pd.DataFrame(list_repo_data)
# write as csv
df_repos.to_csv('df_repos1.csv',index=False)


In [111]:
# df_repos1 = pd.DataFrame(list_repo_data)
df_repos1.to_csv('df_repos2.csv',index=False)

In [113]:
df_repos3 = pd.DataFrame(list_repo_data)
df_repos3.to_csv('df_repos3.csv',index=False)

Question answers for project 1

In [138]:
# upload the dataframes
import pandas as pd

df_users = pd.read_csv("users.csv")
# Convert 'created_at' column to datetime 
df_users['created_at'] = pd.to_datetime(df_users['created_at'])


df_repositories = pd.read_csv("repositories.csv")

In [ ]:
1. Who are the top 5 users in Berlin with the highest number of followers? List their login in order, comma-separated.
Users

In [141]:
# Sort users by number of followers in descending order 
sorted_users = df_users.sort_values(by='followers', ascending=False)

# Get the top 5 users 
top_5_users = sorted_users.head(5)

# List their logins in order, comma-separated 
top_5_logins = list(top_5_users['login'])
print(top_5_logins)

2. Who are the 5 earliest registered GitHub users in Berlin? List their login in ascending order of created_at, comma-separated.

In [155]:
sorted_users = df_users.sort_values(by='created_at',ascending=True)
earl_user = sorted_users.head(5)
earliest_logins = ','.join(earl_user['login'].tolist())
print(earliest_logins)

schacon,adamwiggins,myobie,lstoll,znarf


3. What are the 3 most popular license among these users? Ignore missing licenses. List the license_name in order, comma-separated.

In [170]:
# filter missing licenses
df_all_lic = df_repositories[df_repositories['license_name'].notnull()]
# df_all_lic.shape
','.join(df_repositories['license_name'].value_counts().head(3).index.tolist())

'mit,apache-2.0,other'

4. Which company do the majority of these developers work at?

In [ ]:
df_all_com = df_users[df_users['company'].notna()]
df_all_com['company'].value_counts()


5. Which programming language is most popular among these users?

In [ ]:
# frequency of languages from repository table
df_repositories['language'].value_counts()

6. Which programming language is the second most popular among users who joined after 2020?

7. Which language has the highest average number of stars per repository?

In [178]:
df_new = df_repositories.groupby(['language']).agg({'login': 'count', 'stargazers_count': 'sum'})

In [180]:
df_new['stra_per_rep'] = df_new['stargazers_count']/df_new['login']
df_new.sort_values(by='stra_per_rep', ascending=False).head(1)

,login,stargazers_count,stra_per_rep
language,,,
Fluent,1,12952,12952.0
